In [1]:
import sys
import scanpy as sc

sys.path.append("../..")
from scripts.runner_models import cv_scvi_5fold

In [4]:
# Load data
IN_PATH_neu = "../../data/raw_count/GSE169569_raw_counts.h5ad"
IN_PATH_cov = "../../data/raw_count/GSE228841_raw_counts.h5ad"

adata_neu = sc.read_h5ad(IN_PATH_neu)
adata_cov = sc.read_h5ad(IN_PATH_cov)

In [5]:
import numpy as np

# Extract data matrices and metadata
X_counts = adata_neu.layers["counts"] # raw counts
gene_names = np.array(adata_neu.var_names, dtype=str)
batches = np.array(adata_neu.obs["BioProject"], dtype=str)
norm_layer = adata_neu.layers["log2_1p_CPM_original"]

In [7]:
import optuna
import numpy as np
import pandas as pd

def scvi_objective(
    trial,
    X_counts, gene_names, batches, norm_layer,
    k_search=3,
    seed=123,
    batch_key="BioProject",
    seurat_layer="log2_1p_CPM_original",
    labels=None, # for sillhouette               
    use_composite_score=False, # composite score(TODO: need to check details)
):
    # ----- search space -----
    hp = dict(
        n_latent    = trial.suggest_categorical("n_latent", [16, 32]),
        n_hidden    = trial.suggest_categorical("n_hidden", [128, 256]),
        n_layers    = trial.suggest_categorical("n_layers", [2]),
        dropout_rate= trial.suggest_categorical("dropout_rate", [0.0, 0.1]),
        max_epochs  = trial.suggest_categorical("max_epochs", [150, 250]),
        lr          = trial.suggest_categorical("lr", [1e-3, 5e-4]),
        weight_decay= trial.suggest_categorical("weight_decay", [1e-6, 1e-5]),
        batch_size  = trial.suggest_categorical("batch_size", [256, 512]),
        n_hvg       = trial.suggest_categorical("n_hvg", [800, 1000, 1500, 2000]),
        use_gpu     = True,
    )
    grid = {k: [v] for k, v in hp.items() if k != "n_hvg"}  # one-point grid

    # ----- run light CV -----
    summary, detailed = cv_scvi_5fold(
        X_counts,
        k=k_search,
        scvi_grid=grid,
        n_hvg=int(hp["n_hvg"]),
        R=1,
        mask_frac=0.10,
        thinning_p=0.10,
        random_state=seed,
        hvg_mode="seurat_v3",
        gene_names=gene_names,
        batches=batches,
        norm_layer=norm_layer,
        batch_key=batch_key,
        seurat_layer_name=seurat_layer,
        labels=None, # Sillhouette
        save_dir=None,
    )

    if len(summary) == 0:
        return -np.inf
    row = summary.iloc[0]   # one-point grid → single row

    if not use_composite_score:
        score = float(row["NB_ll_thin"])
    else:
        alpha = 0.05 
        beta  = 0.01
        ll = 0.5 * (float(row["NB_ll_zero"]) + float(row["NB_ll_thin"]))
        dv = 0.5 * (float(row["NB_dev_zero"]) + float(row["NB_dev_thin"]))
        sil = float(row["Silhouette"]) if ("Silhouette" in row and pd.notna(row["Silhouette"])) else 0.0
        score = ll - alpha * dv + beta * sil

    trial.set_user_attr("scvi_summary", summary.to_dict("records"))
    # trial.set_user_attr("scvi_detailed_head", detailed.head(5).to_dict("records"))

    return float(score)


In [8]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=123))
study.optimize(
    lambda tr: scvi_objective(
        tr, X_counts, gene_names, batches, norm_layer,
        k_search=3, seed=123, batch_key="BioProject",
        seurat_layer="log2_1p_CPM_original",
        labels=None,
        use_composite_score=True
    ),
    n_trials=20,
    gc_after_trial=True
)

print("Best params:", study.best_params)

best = study.best_params
scvi_grid_final = {
    "n_latent":    [best["n_latent"]],
    "n_hidden":    [best["n_hidden"]],
    "n_layers":    [best["n_layers"]],
    "dropout_rate":[best["dropout_rate"]],
    "max_epochs":  [max(300, best["max_epochs"])],
    "lr":          [best["lr"]],
    "weight_decay":[best["weight_decay"]],
    "batch_size":  [best["batch_size"]],
    "use_gpu":     [True],
}
summary_final, details_final = cv_scvi_5fold(
    X_counts,
    k=5,
    scvi_grid=scvi_grid_final,
    n_hvg=int(best["n_hvg"]),
    R=3,
    mask_frac=0.10,
    thinning_p=0.10,
    random_state=123,
    hvg_mode="seurat_v3",
    gene_names=gene_names,
    batches=batches,
    norm_layer=norm_layer,
    batch_key="BioProject",
    seurat_layer_name="log2_1p_CPM_original",
    labels=None,
    save_dir="results/scvi_full"
)


[I 2025-09-17 19:28:37,311] A new study created in memory with name: no-name-eacac1b6-90c2-4034-aaf3-5dd2ba6a5feb


2025-09-17 19:28:37 | [cv_scvi_5fold] start k=3 n_hvg=1000 mode=seurat_v3
2025-09-17 19:28:37 | X_counts: dense  (447, 51777) dtype=int64
2025-09-17 19:28:46 | X_counts: zeros=66.23%, approx_int=True
2025-09-17 19:28:46 | [time] [scVI] fold 1/3 start


/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


2025-09-17 19:29:03 | [scVI] fold 1 | HVG=1000
2025-09-17 19:29:03 | [scVI] fold 1 params={'batch_size': 256, 'dropout_rate': 0.0, 'lr': 0.001, 'max_epochs': 150, 'n_hidden': 256, 'n_latent': 16, 'n_layers': 2, 'use_gpu': True, 'weight_decay': 1e-05}
2025-09-17 19:29:22 | [scVI] fit hp={'n_latent': 16, 'n_hidden': 256, 'n_layers': 2, 'dropout_rate': 0.0, 'max_epochs': 150, 'lr': 0.001, 'weight_decay': 1e-05, 'batch_size': 256, 'batch_key': 'BioProject', 'use_gpu': True}
2025-09-17 19:29:22 | X_train: dense  (298, 1000) dtype=int64
2025-09-17 19:29:22 | [time] [scVI] train start


/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/pytho ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/pytho ...
/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have 

...

Epoch 1/300:   0%|          | 0/300 [00:43<?, ?it/s]
2025-09-17 19:56:52 | [time] [scVI] train end in 0.39s
2025-09-17 19:56:52 | [scVI] predict_mean
2025-09-17 19:56:52 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-17 19:56:52 | [time] [scVI] get_likelihood_parameters end in 0.01s
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-17 19:56:52 | [scVI] predict_mean
2025-09-17 19:56:52 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-17 19:56:52 | [time] [scVI] get_likelihood_parameters end in 0.01s
2025-09-17 19:56:52 | [scVI] predict_mean
2025-09-17 19:56:52 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to 

/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/legacy_api_wrap/__init__.py:82: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  return fn(*args_all, **kw)


2025-09-17 19:57:12 | [scVI] fold 5 | HVG=800
2025-09-17 19:57:12 | [scVI] fold 5 params={'batch_size': 512, 'dropout_rate': 0.1, 'lr': 0.0005, 'max_epochs': 300, 'n_hidden': 128, 'n_latent': 32, 'n_layers': 2, 'use_gpu': True, 'weight_decay': 1e-06}
2025-09-17 19:57:12 | [scVI] fit hp={'n_latent': 32, 'n_hidden': 128, 'n_layers': 2, 'dropout_rate': 0.1, 'max_epochs': 300, 'lr': 0.0005, 'weight_decay': 1e-06, 'batch_size': 512, 'batch_key': 'BioProject', 'use_gpu': True}
2025-09-17 19:57:12 | X_train: dense  (358, 800) dtype=int64
2025-09-17 19:57:12 | [time] [scVI] train start


/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/pytho ...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/pytho ...
/home/ma/ma_ma/ma_minjlee/.conda/envs/scvi/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have 

Epoch 1/300:   0%|          | 0/300 [00:44<?, ?it/s]
2025-09-17 19:57:13 | [time] [scVI] train end in 0.39s
2025-09-17 19:57:13 | [scVI] predict_mean
2025-09-17 19:57:13 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-17 19:57:13 | [time] [scVI] get_likelihood_parameters end in 0.01s
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-17 19:57:13 | [scVI] predict_mean
2025-09-17 19:57:13 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
2025-09-17 19:57:13 | [time] [scVI] get_likelihood_parameters end in 0.01s
2025-09-17 19:57:13 | [scVI] predict_mean
2025-09-17 19:57:13 | [time] [scVI] get_likelihood_parameters start
INFO     Input AnnData not setup with scvi-tools. attempting to 

In [9]:
summary_final

,model,n_hvg,MAE_zero,MSE_zero,MedianL1_zero,NB_ll_zero,NB_dev_zero,MAE_thin,MSE_thin,MedianL1_thin,NB_ll_thin,NB_dev_thin,Silhouette,params
0,scVI,800,469.78063,5.120069e+07,131.914311,-8.288211,7.163435,34.348442,277234.700566,16.194821,-2.384981,2.654838,NaN,"{'batch_size': 512, 'dropout_rate': 0.1, 'lr':..."


In [10]:
details_final

,fold,model,params,n_hvg,MAE_zero,MSE_zero,MedianL1_zero,NB_ll_zero,NB_dev_zero,MAE_thin,MSE_thin,MedianL1_thin,NB_ll_thin,NB_dev_thin,Silhouette
0,1,scVI,"{'batch_size': 512, 'dropout_rate': 0.1, 'lr':...",800,245.930901,4.143159e+06,83.891009,-7.193856,5.045560,18.129593,2.048600e+04,9.324852,-2.144412,2.164265,None
1,2,scVI,"{'batch_size': 512, 'dropout_rate': 0.1, 'lr':...",800,249.075400,4.443176e+06,82.799250,-7.776880,6.092666,17.972455,2.174693e+04,9.276781,-2.193283,2.296809,None
2,3,scVI,"{'batch_size': 512, 'dropout_rate': 0.1, 'lr':...",800,253.503688,5.002068e+06,75.477809,-7.920130,6.529982,17.215016,2.295247e+04,8.672253,-2.224451,2.427724,None
3,4,scVI,"{'batch_size': 512, 'dropout_rate': 0.1, 'lr':...",800,1369.817708,2.384690e+08,340.883532,-9.981423,10.368842,101.163212,1.297898e+06,44.924555,-2.996350,3.702380,None
4,5,scVI,"{'batch_size': 512, 'dropout_rate': 0.1, 'lr':...",800,230.575454,3.946030e+06,76.519956,-8.568766,7.780126,17.261934,2.308976e+04,8.775664,-2.366410,2.683014,None


In [ ]:
# Final retrain/eval using the best HPs from the study
# Uses a larger k, more epochs/R, and writes CSVs.

# Build one-point grid from best params
best_grid = {k: [best[k]] for k in best}

final_summary, final_details = cv_scvi_5fold(
    X_counts,
    k=5,                     # fuller CV
    scvi_grid=best_grid,
    n_hvg=int(1000),         # or tune via Optuna if you prefer
    R=3,                     # more repeats for stability
    mask_frac=0.10,
    thinning_p=0.10,
    random_state=seed,
    hvg_mode="seurat_v3",
    gene_names=gene_names,
    batches=batches,
    norm_layer=norm_layer,
    batch_key="BioProject",
    seurat_layer_name="log2_1p_CPM_original",
    labels=None,
    save_dir="results/scvi_full",   # CSVs will be written here
    heartbeat_path=None,
)

display(final_summary.head())
display(final_details.head())
print("Saved to: results/scvi_full")


In [8]:
final_summary

NameError: name 'final_summary' is not defined